## Kamerabild laden

In [80]:
import pypylon.pylon as py

tlf = py.TlFactory.GetInstance()
cam = py.InstantCamera(tlf.CreateFirstDevice(py.DeviceInfo())) 

cam.Open()
cam.UserSetSelector.SetValue(cam.UserSetDefault.Value)
cam.UserSetLoad.Execute()

# Kameraeinstellungen konfigurieren (z.B. Auflösung, Belichtungszeit usw.)
cam.PixelFormat.Value = "BGR8"
cam.ExposureTime.Value = 15000  # Belichtungszeit (in Mikrosekunden)
cam.LightSourcePreset.Value = "Daylight5000K" # RGB Balance Ausgleich
cam.AcquisitionFrameRateEnable.SetValue(True)
cam.Width.Value = cam.Width.Max  # 1280
cam.Height.Value = 960 
cam.CenterX.SetValue(True)
cam.CenterY.SetValue(True)

# Kamerastream starten
if not cam.IsGrabbing():
    cam.StartGrabbing(py.GrabStrategy_LatestImageOnly)
    # Bild von Kamera erfassen
    grab_result = cam.RetrieveResult(5000, py.TimeoutHandling_ThrowException)
    if grab_result.GrabSucceeded():
        image = grab_result.Array
        grab_result.Release()

# Kamerastream stoppen und Ressourcen freigeben
if cam.IsGrabbing():   
    cam.StopGrabbing()
    cam.Close()


### Bild anzeigen

In [66]:
import cv2
import numpy as np

def show_image(image: np.ndarray) -> None:
    while True:
        cv2.imshow('Bild', image)
        if ( cv2.waitKey(30)  == 27 ):
            break

    cv2.destroyAllWindows()

In [91]:
show_image(image)

In [22]:
#cv2.imwrite('test.jpg', image)

True

### Bild Vorberarbeiten

In [82]:
import numpy as np

# Konvertieren in Graustufen (für die Kreis-Erkennung)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Finde Kreise im Bild mit Hough-Transformation
circles = cv2.HoughCircles(
    gray, 
    cv2.HOUGH_GRADIENT, 
    dp=1.5,  # Inverser Auflösungsfaktor
    minDist=100,  # Minimale Distanz zwischen den Zentren der Kreise
    param1=300,  # Parameter für den Canny-Edge-Detektor (HoughCircles)
    param2=30,   # Schwellenwert für die Kreiserkennung
    minRadius=250,  # Minimale Kreisgröße
    maxRadius=350   # Maximale Kreisgröße
)

# Überprüfen, ob Kreise gefunden wurden
if circles is not None:
    # Konvertiere die Koordinaten und den Radius in Ganzzahlen
    circles = np.round(circles[0, :]).astype("int") 

    # Nehme den ersten gefundenen Kreis (Annahme: es gibt nur einen Kreis)
    (x, y, r) = circles[0]
    
    # Festgelegter Radius zum Zuschneiden
    #r = 208
    #r = 400

    if False:
        # Kreis auf Bild einzeichnen
        cv2.circle(image, (x, y), r, (0,255,0), 4)

    r = 350

    # Sicherstellen, dass der Ausschnitt im Bildbereich liegt
    if y - r >= 0 and y + r <= image.shape[0] and x - r >= 0 and x + r <= image.shape[1]:
        # Schneide das rechteckige Stück um das Zentrum des Kreises aus
        cropped = gray[y - r:y + r, x - r:x + r]
else:
    print('Keine Kreise gefunden')

Jetzt schauen wir uns das Bild an und achtend darauf, das....


Ansonsten muss ich folgende Parameter in der obigen Funktion anpassen:
- maxRadius


In [90]:
show_image(cropped)

In [85]:
cropped.shape

(700, 700)

In [88]:
cropped = cv2.resize(cropped, (416, 416))

In [89]:
cv2.imwrite('test.jpg', cropped)

True